# LlamaCPP 

In this short notebook, we show how to use the [LlamaCPP python](https://github.com/abetlen/llama-cpp-python) library with LlamaIndex.

We use the [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) model by default, along with the proper prompt formatting.

## Installation

To get the best performance out of `LlamaCPP`, it is recomended to install the package so that it is compilied with GPU support. A full guide for installing this way is [here](https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal).

Full MACOS instructions are also [here](https://llama-cpp-python.readthedocs.io/en/latest/install/macos/).

In general:
- Use `CuBLAS` if you have CUDA and an NVidia GPU
- Use `METAL` if you are running on an M1/M2 MacBook
- Use `CLBLAST` if you are running on an AMD/Intel GPU

## Setup LLM

The LlamaCPP llm is highly configurable. Depending on the model being used, you'll want to pass in `messages_to_prompt` and `completion_to_prompt` functions to help format the model inputs.

Since the default model is llama2-chat, we use the util functions found in [`llama_index.llms.llama_utils`](https://github.com/jerryjliu/llama_index/blob/main/llama_index/llms/llama_utils.py).

For any kwargs that need to be passed in during initialization, set them in `model_kwargs`. A full list of available model kwargs is available in the [LlamaCPP docs](https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#llama_cpp.llama.Llama.__init__).

For any kwargs that need to be passed in during inference, you can set them in `generate_kwargs`. See the full list of [generate kwargs here](https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#llama_cpp.llama.Llama.__call__).

In general, the defaults are a great starting point. The example below shows configuration with all defaults.

In [2]:
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin",
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama.cpp: loading model from /Users/loganmarkewich/Library/Caches/llama_index/models/llama-2-13b-chat.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 3900
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: mem required  = 7632.72 MB (+ 3046.88 MB per state)
llama_new_co

We can tell that the model is using `metal` due to the logging!

## Start using our `LlamaCPP` LLM abstraction!

In [4]:
response = llm.complete("Hello! Can you tell me a poem about cats and dogs?")
print(response.text)

  Of course! Here's a fun little poem about cats and dogs:

Cats and dogs, so different yet the same,
Both furry friends, with their own special game.

Cats purr and curl up tight,
Dogs wag their tails with delight.

Cats chase mice with stealthy grace,
Dogs bark and chase with joyful pace.

But when the day is done,
Both cats and dogs find comfort in a warm embrace.

So here's to our feline and canine friends,
Both equally loved, until the very end.

I hope you enjoyed that little poem! Do you have any other questions or requests?



llama_print_timings:        load time =  8161.74 ms
llama_print_timings:      sample time =   113.45 ms /   162 runs   (    0.70 ms per token,  1427.97 tokens per second)
llama_print_timings: prompt eval time =  8161.68 ms /    61 tokens (  133.80 ms per token,     7.47 tokens per second)
llama_print_timings:        eval time =  6929.98 ms /   161 runs   (   43.04 ms per token,    23.23 tokens per second)
llama_print_timings:       total time = 15406.04 ms


In [5]:
response_iter = llm.stream_complete("Can you write me a poem about fast cars?")
for response in response_iter:
    print(response.delta, end="", flush=True)

Llama.generate: prefix-match hit


  Sure thing! Here's a poem about fast cars:

Fast cars, oh how they thrill
With their sleek designs and powerful bills
They race down the road, a blur of speed
Leaving all else in their dusty need

Their engines purr, their tires squeal
As they zip through the streets, it's a real deal
The wind rushes by, a roar of sound
As they leave all others in the ground

With their shimmering paint and sleek lines
They're a sight to behold, oh so fine
They race and glide with graceful ease
Fast cars, oh how they please

So here's to the fast cars, a poem of praise
For the thrill and joy they bring to our days
May their engines roar and their wheels spin
Forever and always, let them win.


llama_print_timings:        load time =  8161.74 ms
llama_print_timings:      sample time =   160.54 ms /   201 runs   (    0.80 ms per token,  1252.01 tokens per second)
llama_print_timings: prompt eval time =  1127.02 ms /    14 tokens (   80.50 ms per token,    12.42 tokens per second)
llama_print_timings:        eval time =  6295.21 ms /   200 runs   (   31.48 ms per token,    31.77 tokens per second)
llama_print_timings:       total time =  7947.75 ms
